## Import Libraries

In [193]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import pyperclip

import pandas as pd
import time

AttributeError: type object 'pandas._libs.groupby.array' has no attribute '__reduce_cython__'

### Clipboard copy

In [178]:
def clipboard_input(driver, user_input):
    # save existing clipboard data
    existing_clipboard = pyperclip.paste()

    # copy user_input to clipboard. then paste it
    pyperclip.copy(user_input)
    ActionChains(driver).key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()

    # recover the original clipboard data
    pyperclip.copy(existing_clipboard)  
    time.sleep(1)

### Load user info

In [179]:
def load_user_info():
    f = open('user_info.txt', 'r')
    user_info = f.readlines()
    f.close()
    
    return user_info[0][:-1], user_info[1][:-1]

### Naver Login

In [180]:
def login_naver(driver, user_id, user_pw):
    # naver login page
    driver.get('https://nid.naver.com/nidlogin.login?svctype=262144&url=http://m.naver.com/aside/')

    # input id 
    driver.find_element_by_xpath('//*[@id="id"]').click()
    clipboard_input(driver, user_id)

    # input pw
    driver.find_element_by_xpath('//*[@id="pw"]').click()
    clipboard_input(driver, user_pw)

    # click login btn
    driver.find_element_by_xpath('//*[@id="frmNIDLogin"]/fieldset/input').click()
    time.sleep(2)

    # click 
    driver.find_element_by_xpath('//span[@class="btn_cancel"]').click()

### Enter Details to search in Joonggonara

In [181]:
def enter_details(driver, item = "에어팟", keywords_and = "미개봉", keywords_not = "프로", keywords_or = "유선", sentence = ""):
    # click detail search btn
    driver.switch_to.frame(driver.find_element_by_name("cafe_main"))
    driver.find_element_by_xpath('//*[@id="detailSearchBtn"]').click()
    
    # enter details
    driver.find_element_by_xpath('//input[@placeholder="검색어를 입력해주세요"]').send_keys(item)
    driver.find_element_by_xpath('//input[@placeholder="다음 단어 모두 포함"]').send_keys(keywords_and)
    driver.find_element_by_xpath('//input[@placeholder="다음 단어 제외"]').send_keys(keywords_not)
    driver.find_element_by_xpath('//input[@placeholder="다음 단어 중 1개 이상 포함"]').send_keys(keywords_or)
    driver.find_element_by_xpath('//input[@placeholder="다음 어절, 어구 정확히 일치"]').send_keys(sentence)

    driver.find_element_by_xpath('//form[@name="frmSearchTop"]/div[@class="input_search_area"]/button[@class="btn-search-green"]').click()


#### Run ChromeDriver

In [182]:
driver = webdriver.Chrome()
driver.implicitly_wait(2)

#### Login Naver

In [183]:
user_id, user_pw = load_user_info()
login_naver(driver, user_id, user_pw)

#### Log onto a Joonggonara

In [184]:
# log onto site. to search item
driver.get('https://cafe.naver.com/joonggonara?iframe_url=/ArticleSearchList.nhn%3Fsearch.clubid=10050146%26search.searchBy=0')


#### Search items and get boards

In [185]:
# log onto Joonggonara and enter details
driver.get('https://cafe.naver.com/joonggonara?iframe_url=/ArticleSearchList.nhn%3Fsearch.clubid=10050146%26search.searchBy=0')
enter_details(driver)

In [187]:
board_navigator = driver.find_element_by_css_selector('div.prev-next')
time.sleep(5)
board_list = board_navigator.find_elements_by_tag_name('a')
board_href = board_list[1].get_attribute('href')[:-1]
board_idx = 0

#### Get Posts Links

In [188]:
post_link_list=[]
while True:
    # change board
    board_idx+=1
    driver.get(board_href+str(board_idx))
    driver.switch_to.frame(driver.find_element_by_name("cafe_main"))
    
    # get posts
    post_list = driver.find_elements_by_class_name('article')
    for post in post_list:
        post_link_list.append(post.get_attribute('href'))
    
    # terminate condition
    if len(post_list) == 0:
        break

KeyboardInterrupt: 

#### Get Price Info

In [189]:
def get_post_info(driver, href):
    driver.get(href)
    driver.switch_to.frame('cafe_main')
    soup = bs(driver.page_source, 'html.parser')
    
    # post_title
    title = soup.select('div.tit-box span.b')[0].get_text()
    
    # merge contents to single text
    content_tags = soup.select('#tbody')[0].select('p')
    content = ' '.join([ tags.get_text() for tags in content_tags ])
    
    return {'title' : title, 'content' : content}

In [190]:
post_info = get_post_info(driver, post_link_list[0])

NameError: name 'bs' is not defined